# Bllossom-8B 양자화 모델을 이용한 한국어 LLM 튜토리얼

## 01. 활용할 package 설정
 - GPU사용하기: colab에서 런타임 --> 런타임유형변경 --> T4 선택
 - 패키지설치: 아래 pip를 이용해 Transformers, accelerate 설치
 - 런타임재시작: 런타임 --> 세션다시시작  (accelerate설치 시 런타임 다시시작하셔야됩니다!)

In [7]:
!pip install transformers==4.40.0 accelerate
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python #양자화 구동을 위한 Llama C++ 설치
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./' #Bllossom모델 다운로드

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 34.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [8]:
!pip install transformers==4.40.0 accelerate
!pip install llama-cpp-python

In [9]:
from llama_cpp import Llama
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## 02. 모델준비

In [10]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Llama(
    model_path='./llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=10000,
    n_gpu_layers=-1        # Number of model layers to offload to GPU
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./llama-3-Korean-Bllossom-8B-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 145088
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.f

## 03. 추론

In [20]:
PROMPT = \
'''
Role: "You are a teacher running a college entrance essay interview academy."
Task: "Your main job is to generate example questions from a set of example passages. The number of passages in the set and the number of questions are not fixed. I will show you an example set of passages and example questions. Based on this, you will generate questions from a new set of passages in a similar manner. Your task is to generate only the questions."
'''

instruction = 'Did you understand what I said?'

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":512,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.2,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     559.31 ms
llama_print_timings:      sample time =     174.61 ms /    79 runs   (    2.21 ms per token,   452.44 tokens per second)
llama_print_timings: prompt eval time =     282.55 ms /   111 tokens (    2.55 ms per token,   392.85 tokens per second)
llama_print_timings:        eval time =    2309.65 ms /    78 runs   (   29.61 ms per token,    33.77 tokens per second)
llama_print_timings:       total time =    2849.59 ms /   189 tokens


Yes, I understood that you have a job as a teacher running a college entrance essay interview academy. Your main job is to generate example questions from a set of example passes. You will show me an example set of passes and example questions, and then I will help you generate questions from a new set of passes in a similar manner. I will only provide the questions and not the answers.


In [31]:
PROMPT = \
'''
Here is a sample set of prompts given as an example. This set consists of four prompts.

[제시문 (가)]
예술은 새롭게 느끼고 생각하게 함으로써 인간과 현실과 세계를 신선하게 경험하도록 한다. 예술의 언어는
인간을 폭넓게 이해하며, 현실의 숨은 모습을 드러내고, 세계의 깊이를 자각하게 한다. 기존의 것을 있는 그
대로 받아들이지 않고 새롭게 이해하는 것은 대상의 변형 가능성을 찾아내는 것이다. 이러한 변형 가능성은
새 세계를 개시(開示)할 수 있는데, 바로 이 점이 보편성에 대한 예술의 대응 방식이 철학, 사회과학 혹은
자연과학과는 다르다는 것을 보여준다. 예술은 처음부터 보편성이나 전체성을 말하지 않는다. 그렇듯이 ‘미
(美)’ 혹은 ‘인간성’을 직접 말하지 않는다. 프로파간다 예술1)이나 사회주의 리얼리즘2)에서는 이를 직접 말
하기도 하지만, 이런 종류의 작품이 갖는 한계는 분명하다. 옳고 선하고 아름다운 일과 좋은 이념은 실현되
기 어렵기 때문에 버거운 것이다. 자유와 정의, 평등과 인권 같은 보편적인 가치들은 시종일관 반성되지 않
으면 거짓이기 쉽다. 검토와 비판을 허락하지 않는 보편성은 가짜이다. 참된 보편성은 개별적 경험과 특수한
사례의 지속적 검토를 견뎌낼 수 있어야 한다. 처음부터 보편타당한 원리를 상정하는 것이 아니라, 이 원리
의 갱신 가능성을 구체적 경험 속에서 늘 다시 검토해야 한다. 보편성의 검토 작업에서 예술의 위치, 역할,
메커니즘은 독특하다. 그 이유는 첫째, 예술은 개념적인 추상이 아니라 구체적인 경험에 천착하고, 둘째, 이
를 보편성의 지평 안으로 유도하여 재해석하는 활동이기 때문이다. 예술은 구체적 보편성 혹은 개별적 일반
성의 차원으로 나아간다. 특히 감각적인 경험과 구체적인 현실에 대한 밀착이야말로 예술의 제1덕목이다. 이
러한 밀착을 통해 예술은 흩어지고 무너지고 짓밟히고 외면된 것을 새롭게 읽어내고자 한다. 예술의 저항은
선언의 낭독이나 거리에서의 궐기 같은 직접적인 방식이 아니며, 설명이나 진단 혹은 분석과 같은 사회과학
적 방식 또한 아니다.
그러나 분명한 것은 예술이 단순히 운명의 필연성에 복속되지 않고 그 나름으로 대응하고 있다는 점이다.
예술은 드러나지 않을 수 있지만 미세하고도 오랜 파문을 일으킨다는 점에서 가장 강력한 저항매체라고 할
수 있다. 예술은 실천적이지만 행동주의적인 것이 아니며, 성찰적이지만 사변적인 것은 아니다. 심미적 개입
은 사유와 실천 사이에서 감각적인 경험과 구체적인 현실에 밀착하는 가운데 부단히 이루어진다. 그런 점에
서 예술은 운명의 악순환 고리를 끊으면서 우리를 더 넓고 깊은 행복으로 이끌고 나아간다고 말할 수 있다.
예술작품을 놓고 그것이 뜻하는 바에 대한 진위나 가치를 따질 수는 없다. 그런 방식으로 예술작품을 대한
다면, 그것은 이미 예술작품이 아닌 것이다. 시나 소설과 같은 예술작품은 일상적으로 사용되는 언어와 달
리, 객관적 사실을 표상하거나 구체적인 감정이나 생각을 표현하는 것도 아니고, 행위를 유도하기 위해 쓰이
지도 않는다. 따라서 한 예술작품에 대해 말할 때, 예술가의 의도나 생각과 같은 비언어적 맥락은 예술작품
의 의미를 결정하는 데 도움이 되지 않는다. 한마디로 말해 예술작품의 의미는 예술가의 의도일 수 없다.

[제시문 (나)]
예술작품이란 인간의 가치를 표현하고 형상화한 하나의 형식이다. 이 추상화된 형식은 관객에게 즐거움을
줄 수 있지만, 형식을 단순히 추상적 도안으로 해석해서는 안 된다. 형식은 표상적이고 재현적이며 함축적인
요소의 결합체이다. 형식을 이렇게 해석할 때 예술에 적용되는 의미가 풍부해지고 인간의 가치 위계에서 당
연히 높은 위치를 점하게 된다. 많은 예술가가 자신을 위해 작품을 만든다고 하지만, 사실은 자신의 규범을
설정하고 있는 것이다. 예술작품은 사회적인 반응과 수렴을 요구하는 관계 속에서 비로소 의미를 가진다. 예
술가는 예술 경향의 여러 갈래 속에서 자신만의 고유한 작업을 통해 인성을 창조하며, 인간적 고뇌와 고통
을 풀어보려 노력한다. 이를 통해 예술가는 현세의 이상향을 찾아 나서기도 한다. 실제로 모든 종류의 가치,
이를테면 도덕적, 종교적, 경제적, 과학적 가치 등은 적절한 환경 아래에서 미적으로 관조된다. 이러한 기초
위에서 예술이 인간의 모든 가치 영역을 표현하거나 포함한다고 볼 수 있다. 예술이 직접적인 혹은 특별한
의무를 명시하지는 않더라도, 그 영향력으로 우리가 언제나 고귀한 존재로 남을 수 있게 해준다.
예술은 인간을 감성적, 지적, 도덕적으로 성장시킨다는 측면에서 교육적 기능을 담당한다고 볼 수 있다. 시
나 소설과 같은 문자 예술은 독자에게 과학으로는 결코 도달할 수 없는 인간에 대한 진실성을 보여주고, 사
회나 역사에 대한 의식과 지식의 폭을 넓혀주며, 도덕적 감수성을 고양하는 데 크게 기여한다. 위대한 소설
을 읽었을 때, 훌륭한 연극을 보았을 때, 우리는 흔히 커다란 정신적 충격을 경험한다. 도스토옙스키의 「죄
와 벌」을 읽거나 셰익스피어의 「리어왕」을 관람한 후에 우리는 그동안 알아왔던 세계가 뒤바뀌고 뒤집히
는 느낌을 가질 수 있다. 이런 작품들은 우리가 지금 살고 있는 세계 속에 안주하는 것을 용납하지 않는다.
또한 세계를 새롭게 볼 수 있게 하고, 새로운 감각으로 우리 자신을 느끼게 하며, 행위의 의미를 새로운 시
각에서 조명하게 한다. 예술을 통한 이러한 경험은 비단 문학작품에서만 나타나는 것은 아니다. 위대한 그림
을 보았을 때, 멋진 무용을 구경했을 때, 훌륭한 음악을 들었을 때도 마찬가지이다. 예술작품과의 접촉을 통
해 우리는 사물 현상을 다른 차원에서 신선하게 느끼며, 우리의 행위를 새로운 도덕적 척도에서 평가하게
되는 것이다. 이를 통해 우리는 세계와 우리 자신이 달라진 것을 발견할 수 있다.

[제시문 (다)]
음악은 인간의 마음을 선하게 한다. 평화로운 음악은 사람으로 하여금 화목하고 상호 공경하게 하여 방탕
에 빠지지 않도록 하며, 엄숙하고 장엄한 음악은 마음을 가지런하게 하여 어지럽지 않게 한다. 그러므로 마
음을 감동시키고 풍속을 바꾸는 데에는 음악보다 더 좋은 것이 없다. 음악은 사람 각자의 심성을 교화시키
고, 나아가 사회 구성원 간의 조화로움을 지향한다. 이를 유가(儒家)에서는 예교(禮敎)와 악교(樂敎)를 통합
하여 ‘예악교화(禮樂敎化)’라고 이른다. 예악(禮樂)은 사람들이 도덕적 선(善)을 성취하는 도덕 수양에서 중
요한 수단으로 간주된다. 여기서 예(禮)는 이지(理智)의 측면에서 말한 것이고, 악(樂)은 감정의 측면에서
말한 것이다. 이지는 차이를 구별하는 것이고, 감정은 화합하는 것이다. 그러므로 예의는 인륜 관계를 구별
하여 각자 자신의 위치를 편안히 여기고 그 직분을 담당하게 하며, 음악은 집단의 감정을 조화롭게 하고 의
사소통을 통하여 마음을 융합하게 만든다. 어느 학자는 “음악은 인간의 감정을 화합하며, 예는 인륜 관계의
차이를 구별한다”고 지적했는데, 이것은 예악의 상대적 역할과 조화를 말한 것이다. 또 『예기(禮記)』에서
는 “음악은 안에서부터 나오며, 예는 밖에서부터 생겨난 것이다”라고 하였다. 이처럼 예와 악이 하나는 안에
서, 하나는 밖에서 상호 조절하여 이지와 감정 모두 평정을 얻게 된다. “음악이 잘 통하면 원망이 없고 예가
잘 통하면 다툼이 없으니, 서로 양보하여 천하를 다스리는 것은 예악을 두고 한 말이다”라는 구문에서 이를
확인할 수 있다. “예악을 통합하여 인간의 마음을 다스린다”는 것은 예악의 효용을 의미한다. ‘마음을 다하여
변화를 극진하게 하는 것’은 음악이 마음 깊숙한 곳을 울려 감정의 변화를 극진히 하는 것을 뜻하며, ‘성실
함을 밝히고 허위를 제거하는 것’은 예가 공경의 마음을 표현하고 거짓 행위를 제거하는 것을 말한다. 그래
서 “음악이 행해지면 뜻이 맑아지고, 예를 닦으면 덕행이 이루어진다”, “예에서 사람이 서고, 악에서 사람이
완성된다”는 등의 말은 예술과 도덕의 조화로운 관계 속에서 도덕적으로 완성된 인격체로서 거듭날 수 있음
을 강조한 것이다. 이와 더불어, “음악이 나쁜 풍속을 좋은 방향으로 변화시키니 천하가 모두 태평해진다”라
는 말은 예술이 결코 인간의 공동체적 삶과 분리될 수 없으며, 구성원의 삶을 인격적으로 고양하고 공동체
의 안녕과 발전에 기여해야 한다는 것이다. 결국 ‘예술을 위한 예술’은 존재할 수 없으며, 개인적 차원이나
공동체 차원에서 선을 추구하고 이를 지향하는 경우에만 진정한 예술이 된다.

[제시문 (라)]
예술은 의미의 문제가 아니라 감각적인 질과 구상(構想)의 문제이다. 예술의 재료가 의미중립적인 것이든
인간적인 것이든, 우리는 예술의 재료와 이 재료로 만들어진 예술작품을 구분해야 한다. 예술작품의 창조나
감상에 대한 관심은 재료 자체에 있는 것이 아니라 재료로 만들어진 결과물에 있다. 그러나 그 모두가 바로
예술작품이 될 수는 없으며, 사람들이 흥미나 감흥을 느낄 때에만 비로소 ‘미적’이라고 할 수 있다. 예술의
관심이 사회의 규칙과 원리에 포함된다는 시각이나 이러한 원리를 예술에 적용하려는 시도는 미적 창조와
감상 모두에 유해한 것이다. 미적 대상은 인식적인 가치로부터 분리되어 있으며, 감각적이거나 형식적인 성
질 자체를 깨달았을 때의 순수한 기쁨에서 정당성을 얻는다. 아름다운 사물에서 ‘아름다움’의 의미를 찾아내
고 받아들이는 것은 교양 있고 선택된 사람의 특권이다. 셰익스피어의 작품은 운문(韻文)의 수려함, 비유적
표현의 다양함과 풍부함, 그리고 구성의 탁월함 때문에 상찬을 받는다. 예술가의 창조성은 그 자체로 목적이 된다.
그렇다면 예술은 무엇에 도움을 줄 수 있는가? 상업주의는 도구적 문화를 요구하며 예술의 유용성만을 강
조하기 때문에 비판에 직면하고 있다. 음미되는 가치 자체만으로 예술은 존재의 이유가 있다. 예술은 절대로
어떤 것에도 도움을 주지 않으며, 역사, 사회, 그리고 개인의 맥락과는 단절된 자율적 유기체이다. 계산될 수
있는 한에서 세계는 과학의 지배를 받기 마련이지만, 예술의 왕국은 ‘계산될 수 없는 곳’에서 시작된다. 예술
은 매 순간 자기 충만성을 향유하게 하며, 경험을 생기 있게 하고, 경험이 지닌 상상적이고 정서적인 기쁨을
즐기게 한다. 좋은 삶이란 경험의 매 순간을 풍부하고 정열적으로 사는 것이다. 그러므로 예술을 사랑하는
것이야 말로 좋은 삶의 원천이 된다.


Next, I will show you the example question. This question was generated from the sample set of prompts above.


[예시 질문]
제시문(가) ∼ 제시문(라)는 예술에 관한 견해를 담고 있다. 제시문들을 상반된 두 입장으로 분류하고 각 입장을 요약하시오.

In this way, the example question references the content of the sample prompts, summarizing them or asking about topics that can be considered from them. When creating question, it's important to consider the relationships between the contents of each prompt.
'''

instruction = '''
Please create one question and one answer based on the following prompts. You need to generate questions from the [new prompts]. When generating question, the content should not imply or suggest the answers. Please delete any unnecessary sentences. The format should be as follows:

["Question 1"
"Question 2"
"Question 3"
"Answer 1"
"Answer 2"
'Answer 3"]

Please create one question and one answer. As I said, when generating question, the content should not imply or suggest the answer. When creating question, it's important to consider the relationships between the contents of each prompt. The question should be in Korean, and concise, consisting of 1-2 sentences.


[new prompts (가)]
협력이 가능한 일반적인 요건은 두 가지이다. 첫째, 협력은 호혜주의를 바탕으로 한다. 둘째, 호혜주의가 오
랜 기간 동안 안정적으로 유지될 수 있다는 믿음이 있어야 한다. 결국 서로가 상대방을 도울 수 있을 때 협
력이 가능하다. 그런데 도울 때 대가를 치러야 한다면 딜레마가 생길 수 있다. 협력 당사자의 입장에서 보면
상대방의 협력 덕택에 얻는 이득이 내가 협력할 때 치러야 할 비용보다 더 커야만 협력할 이유가 생겨서 상
호 이득의 기회를 현실화할 수 있다. 이 경우 협력 당사자 모두가 상호 협력을 상호 배반보다 더 선호하게
되는 것은 당연하다. 그러나 내가 원하는 것을 얻기가 쉽지는 않다. 다음 두 이유 때문이다. 첫째, 나는 도움
을 주지 않는 것이 단기적으로 더 이득이지만 그럼에도 불구하고 상대방은 나를 돕도록 유도하고 싶어 한
다. 둘째, 나는 남에게 큰 비용이 드는 도움을 제공하지 않으면서도 받을 수 있는 도움은 모두 받고 싶은 유
혹을 느낀다. 하지만 일단 한 집단 안에서 호혜주의를 바탕으로 협력이 자리를 잡으면 어떤 비협력적인 전
략도 침범하기 어려워서 스스로를 지켜낼 수 있다.
협력이 배태될 수 있고, 온갖 다양한 전략이 뒤섞여 있는 환경에서도 번성할 수 있으며, 또 일단 자리를 잡
은 뒤에는 스스로를 보호할 수 있다는 사실은 매우 고무적이다. 하지만 무엇보다 흥미로운 사실은 이러한
결과를 얻는데 개인이나 사회적인 환경의 특성은 거의 영향을 미치지 못한다는 점이다. 개인은 논리적일 필
요가 없고 어떻게 하고 왜 그렇게 해야 하는지 알지 못해도 된다. 진화 과정은 성공적인 전략들이 자연적으
로 번성하게 해 준다. 또한 개인은 서로 메시지나 약속을 주고받을 필요도 없다. 말도 필요하지 않다. 행동
이 말을 대신하기 때문이다. 마찬가지로 개인 사이에 어떤 신뢰도 필요하지 않다. 호혜주의만으로 충분히 배
반을 비생산적이 되게 할 수 있기 때문이다. 이타주의도 필요하지 않다. 성공적인 전략은 이기주의자한테서
도 협력을 이끌어내기 때문이다.
협력의 창발과 성장, 유지에 꼭 필요한 개인과 사회 환경에 대한 가정이 몇 가지 있기는 하다. 우선 예전에
상호 작용했던 협력자를 알아볼 수 있어야 하며 상대와의 과거 상호 작용 내력을 기억할 수 있어야 한다. 그
래야 그에 따라 반응할 수 있다. 하지만 실제로는 인식과 기억의 요건을 맞추는 것이 그렇게 어려운 것은 아
니다. 박테리아조차도 단 하나의 상대하고만 상호 작용한다거나, 단 하나의 전략을 가지고 상대방의 가장 최
근 행위에 대해서만 반응하는 식으로 이런 요건을 충족시킨다. 박테리아가 할 수 있으면 사람도 할 수 있다.

[new prompts (나)]
인간의 노력을 조정하는 수단으로 경쟁의 힘을 최대한 잘 활용해야 한다. 이는 유효한 경쟁이 창출될 수
있는 곳에서는 그 어떤 방법보다도 이것이 개별적 노력의 좋은 길잡이가 된다는 확신에 기초한 것이다. 나
는 경쟁이 유익하게 작동하려면 세심하게 배려된 법적 틀이 필요하다는 사실을 부인하지 않는다. 또한 만약
경쟁이 유효하게 이루어지도록 하는 조건을 충족시키기 어려운 경우에는 다른 방법을 경제활동의 길잡이로
삼아야 한다는 것도 부정하지 않는다. 그러나 개인의 개별적 노력을 조정하는 방법으로 더욱 열등한 방법이
경쟁을 대체하는 것에는 반대한다.
그리고 알려진 방법 중 가장 효율적이라는 이유뿐만 아니라 더 크게는 권력의 강제적이고도 자의적인 간섭
없이도 우리의 행위를 조정할 수 있는 유일한 방법이기 때문에 나는 경쟁을 우월한 방법으로 간주한다. 사
실, 경쟁을 선호하는 핵심 이유 중 하나는 의식적인 사회적 통제가 필요하지 않다는 점이며, 특정한 직업이
그 직업과 연관된 불리한 점과 위험 요소를 상쇄하고도 남을 만큼 전망이 있는지 스스로 판가름할 기회를
각자에게 부여한다는 점이다. 경쟁을 사회 조직의 원칙으로 잘 활용하기 위해서는 강제적 간섭을 배제해야
하지만 그 작동을 도와줄 수 있는 특정한 방식의 간섭은 인정할 수 있으며 정부의 적절한 조치가 필요한 경
우도 있다. 그러나 강제력을 사용해서는 안 된다는 점은 특별히 강조해야 한다. 시장 참여자는 거래 상대방
을 찾을 수 있는 한 어떤 가격에서건 자유롭게 팔고 살 수 있어야 하고, 누구든 자유롭게 생산할 수 있고,
팔릴 수 있는 어떤 것도 생산하여 팔고 살 수 있어야 한다.
업종에 상관없이 모든 사람에게 같은 조건으로 진입을 자유롭게 허용해야 한다는 것은 본질적으로 중요하
다. 아울러 개인이나 단체가 공개적인 혹은 드러나지 않은 힘을 이용하여 이러한 진입을 제한하려는 것을
법이 용인하지 않아야 한다는 것 또한 본질적으로 중요하다. 특정 상품에 대해 가격이나 물량을 통제하게
되면 각자의 노력을 유효하게 조정하는 경쟁의 능력은 박탈된다. 왜냐하면 이 경우 가격의 변화가 변화된
모든 상황을 더 이상 반영하지 못하게 되며, 그 결과 가격의 변화가 더 이상 개인의 행위를 나타내는 믿을
만한 길잡이가 될 수 없기 때문이다.

[new prompts (다)]
전체 사회 구성원이 공동으로 소유한 산림과 바다 같은 공유 자원은 배제성은 없지만 경합성이 있다. 배제
성이 없으므로 대가를 지불하지 않고도 누구나 공유 자원을 소비할 수 있는 반면, 경합성이 있으므로 누군
가가 공유 자원을 소비하면 다른 사람의 소비 기회는 감소한다. 따라서 우리는 공유 자원을 다른 사람보다
먼저 더 많이 소비하려고 경쟁한다. 이와 같은 특성 때문에 공유 자원은 쉽게 남용되어 필요한 양보다 과다
하게 소비하게 되고 고갈될 가능성이 커진다. 이에 대해 일부 공동체주의자는 공동체적 해법이야말로 인간
의 본성에 가장 걸맞은 자연스러운 해법이라고 주장한다. 이들에게 자연은 “적자생존” 혹은 “피로 물든 이빨
과 발톱” 등으로 묘사되어온 생존 경쟁의 장이 아니다. 오히려 이들은 상호 부조의 감정이야말로 수십만 년
에 걸친 집단생활을 통해 그리고 지난 수천 년의 사회생활을 통해 배양된 것이며, “전쟁터에서처럼 사람들이
미쳐 돌아가는 상태가 아니라면” 상호 부조의 감정은 거스를 수 없는 법이라고 주장한다.
그러나 인간은 언제나 공공의 이득을 위해서 행동할 준비가 되어 있는 존재는 아니다. 그렇기 때문에 인간
본성에 기댈 수만은 없으며 적절한 유인이 필요하고 적절한 제도가 필요하다. 적절한 제도 하에서는 자신의
이익과 공동의 이익을 조화시킬 수 있을 것이기 때문이다. 이때 적절한 제도란 물질적 이득을 추구하는 개
인을 외적으로 그리고 위에서 아래로 강제하는 장치가 아니라 상호 신뢰를 통해 서로를 규제할 수 있는 견
제 장치이다. 이것이 가능한 것은 인간은 협조하려는 상대방의 의지가 확인되면 언제든지 이에 협조로 응답
하고, 자신의 행동이 타인에게 미치는 영향을 고려하며, 다른 누군가의 행동이 타인에게 해를 입히는 경우
자기 일처럼 나서서 이를 제어하려는 의향을 가진 존재이기 때문이다.
1960년대 오스트레일리아에서 바다가재를 잡는 지역의 어부들은 바다가재의 숫자가 계속 줄어드는 문제에
직면해 있었다. 문제를 해결하기 위해 어부들은 좋은 생각을 해냈다. 공동체를 결성해서 설치할 수 있는 어
망의 숫자를 제한하기로 한 것이다. 이에 다른 사람이 어망을 더 설치하는 것을 서로 감시하게 되면서 불법
적으로 어망을 설치하는 어민이 사라졌고 결국 바다가재의 숫자를 늘리는 데 성공했다. 만약 정부에서 일방
적으로 어망을 제한했다면 여러 나라에서 그랬던 것처럼 일부 어부가 경비정 몰래 바다가재를 제한 없이 잡
다가 경비정이 쫓아오면 잡히지 않기 위해 그물을 끊고 도망가는 불법 조업이 기승을 부렸을 것이다. 그런
데 어부들이 자율적으로 제한을 두면서 자연스럽게 불법 조업이 자취를 감추었다.
공동체를 토대로 사회질서를 유지할 수 있다는 생각을 누군가는 낡은 이념으로, 다른 누군가는 유토피아적
이념으로 여긴다. 그러나 많은 경우 공동체 구성원이 상호 감시하고 상호 제재하는 것이 공유 자원을 관리
하는 효과적인 방법이 된다. 공동체 구성원은 외적 권위체가 갖지 못하는 정보를 갖고 있으며, 공동체 내에
서 상호 신뢰를 기초로 서로를 규제해가면서 비극을 피할 수 있기 때문이다.

[new prompts (라)]
경쟁을 통해 각자의 능력이 최대한 발휘되기도 하지만 경쟁에서 진 쪽은 여러 가지 부작용을 겪기도 하는
것이 사실이다. 경쟁이란 본질적으로 ‘차가운 것’이기 때문이다. 그래서 시장에서는 경쟁에서 지지 않기 위해
경쟁관계의 두 기업이 일시적으로 협력하거나 다양한 방식의 협력관계를 경쟁 속에 부분적으로 도입하는 역
설적인 경우를 볼 수 있다. 예를 들어 오랫동안 경쟁하다가도 경쟁자에게 부품을 공급하는 협력적 공급자로
위치가 일시적으로 변화할 수 있다. 협력을 도입하여 경쟁의 형태가 간접적인 것으로 변화할 수도 있다. 예
들 들어 한 기업이 다른 두 기업과 각각 독립적으로 파트너십을 맺고 협력하여 양쪽의 파트너 기업들이 서
로 경쟁하는 구도가 만들어 질 수도 있다. 판매에서는 경쟁하지만 연구 및 개발에서는 잠정적으로 협력한다거
나 아예 부서를 나누어서 어떤 부서에서는 협력을, 다른 부서에서는 경쟁을 진행하는 방식으로 부분적 협력이
진행될 수도 있다. 특히 새로운 기술이 등장해서 기술 혹은 산업간 융합이 일어날 때, 그리고 기업 규모가
작아서 경쟁력을 높여야 할 때 경쟁자와 협력하는 현상이 일어나기도 한다. 그럼에도 경쟁 메커니즘은 근본
적으로 기업들이 경쟁에서 우위를 확보하기 위해 혁신에 더 집중하도록 유도한다. 일반적으로 기업들은 독
점보다는 경쟁 상황일 때 연구개발 등 혁신 활동에 더 많이 투자할 유인을 갖기 때문이다.
사회 전체 차원에서 보면 경쟁이 심해질수록 사회생활의 접촉대상이 되는 타인은 실재적 혹은 잠재적 경쟁
대상으로만 간주될 것이며 이러한 사고가 팽배할 때 휴머니즘을 토대로 한 사회적 친화력을 견지하기는 어
렵다. 그래서 경제사상가들은 경쟁이 국부의 증대에 기여하지만 사회적 동질성과 도덕을 잠식한다는 지적을
자주 해 왔다. 이런 문제를 고려할 때 제기되는 과제는 한편으로는 경쟁의 기능을 계속 살리면서 다른 한편
으로는 경쟁이 유발하는 비인도적인 측면을 약화시킬 수 있는 ‘황금의 중도’를 설정하는 일이다. 2차 세계대
전 이후 독일을 위시한 몇몇 서구국가들이 도입한 사회적 시장경제 제도는 바로 그와 같은 사실을 배경으로
형성된 것이었다. 효율성을 증대시키는 시장경제의 기능을 토대로 하면서 동시에 자본주의 사회에 내재하는
차가움이나 사회·경제적 역기능을 극소화하자는 것이다.
최근 들어 과거의 성장제일주의 정책을 수정하여 안정과 형평 내지는 삶의 질을 조화시키겠다고 선언하는
국가를 쉽게 찾을 수 있다. 이를 위해서는 무엇보다도 사회의 기본적 가치에 대한 공감이 형성되지 않으면
안 된다. 그렇지 않을 경우 표출되는 다양한 갈등은 사회적 제어능력을 벗어날 위험이 있기 때문이다. 공존
원리로서의 공정한 경쟁을 통한 진보를 훼손하지 않으면서도 휴머니즘을 기저로 하는 사회적 삶의 실존과
조화되는 규범을 마련하는 토론의 장에 모든 사회 구성원 혹은 이익집단의 참여가 필요한 이유도 여기에 있
다. 최근 폭발적으로 분출되는 다양한 욕구·견해·주장 등은 아마도 새로운 질서창출을 위한 사회적 공감대
형성의 불가피한 진통일 것이다.
'''

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":100000,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.2,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     559.31 ms
llama_print_timings:      sample time =    1011.34 ms /   417 runs   (    2.43 ms per token,   412.32 tokens per second)
llama_print_timings: prompt eval time =    3827.69 ms /  2410 tokens (    1.59 ms per token,   629.62 tokens per second)
llama_print_timings:        eval time =   16850.85 ms /   416 runs   (   40.51 ms per token,    24.69 tokens per second)
llama_print_timings:       total time =   22342.36 ms /  2826 tokens


Here is the question and answer in the format that you wanted.

[질문 1]
제시문(가)와 제시문(라)를 바탕으로, 예술과 경쟁에서의 협력과 경쟁의 장단점을 논해주세요.

[답변 1]
예술과 경쟁에서는 협력과 경쟁이 동시에 존재합니다. 협력은 서로 도움이 되고 상호 보완성도 존재하지만, 경쟁에서는 각자가 상대방을 이길려고 노력하며 차별화와 경쟁력을 추구하는 경향이 있습니다. 이러한 경쟁과 협상의 장단점을 분석하면, 경쟁에서는 혁신적인 전략과 기술, 그리고 마케팅 등을 통해 더욱 높은 성과를 얻을 수 있지만, 협상은 상호 협조와 협력을 통해 문제를 더욱 효과적으로 해결할 수 있습니다. 따라서, 어떤 상황에서 협력이나 경쟁이 더 유리하게 작용하는지에에 따라 선택해야 합니다.

[질문 2]
제시문(나)와 제시문(다)를 바탕으로, 공유 자원의 관리와 관련된 경쟁과 협력의 관계를 논의해주세요.

[답변 2]
공유 자원은 경쟁과 협력의 구도를 변화시키는 중요한 요소입니다. 사람들은 서로 공유된 자원을 활용하기 위해 경쟁적으로 행동하지만, 동시에 그 자원들이 고갈되어 더이상 사용할 수 없게 되면 협력적인 노력으로 이를 대처하려고 합니다. 이러한 증시와 협상의 순환은 자연스러운 인간 본성에 기인하며, 적절한 제도와 유인이 필요한 것입니다. 예를 들어, 어부들의 경우 바다가재의 수를 제한하여 불법적인 조제를 방지하고 자원을 효과적으로 관리했습니다.

[질문 3]
제시문(라)와 제시문(다)를 바탕으로, 경제 시장에서의 경쟁과 협력의 관계를 논의해주세요.

[답변 3]
경제 시장에서는 경쟁이 중요한 기능을 하지만, 동시에 협력과 사회적 연대는 필요한 것입니다. 경쟁은 기업들이 혁신적인 전략과 기술, 그리고 마케팅 등을 통해 더욱 높은 성과를 얻을 수 있도록 유도하며, 동시에 사회적 시장경제가 추구하는 것처럼 자본주의 사회의 모순을 완화시키는 역할을 합니다. 이를 위해 경제 주체들은 서로 협동하여 사회적인 문제를 해결하고, 동시에 